In [4]:
!pip install diffusers protobuf bitsandbytes sentencepiece transformers torch python-dotenv accelerate matplotlib

  Using cached protobuf-5.27.1-cp38-abi3-manylinux2014_x86_64.whl.metadata (592 bytes)
Using cached protobuf-5.27.1-cp38-abi3-manylinux2014_x86_64.whl (309 kB)


In [3]:
!pip install --upgrade jupyter ipywidgets tqdm

  Using cached anyio-4.4.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached httpx-0.27.0-py3-none-any.whl.metadata (7.2 kB)
  Using cached setuptools-70.1.0-py3-none-any.whl.metadata (6.0 kB)
  Using cached jsonschema-4.22.0-py3-none-any.whl.metadata (8.2 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached httpcore-1.0.5-py3-none-any.whl.metadata (20 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
  Using cached attrs-23.2.0-py3-none-any.whl.metadata (9.5 kB)
  Using cached jsonschema_specifications-2023.12.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached referencing-0.35.1-py3-none-any.whl.metadata (2.8 kB)
  Using cached rpds_py-0.18.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 2.7 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 kB 2.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/

In [1]:
import os
from dotenv import load_dotenv
from huggingface_hub import login

load_dotenv()

login(token=os.getenv("HF_TOKEN"))

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/nimda/.cache/huggingface/token
Login successful


In [ ]:
# Note you need to get permission to access this model from HF - https://huggingface.co/stabilityai/stable-diffusion-3-medium and this one from deep floyd - https://huggingface.co/DeepFloyd/IF-II-L-v1.0

In [1]:
import os
from transformers import T5EncoderModel, BitsAndBytesConfig
from diffusers import StableDiffusion3Pipeline
import torch
from PIL import Image
import matplotlib.pyplot as plt

# Configure quantization to load the model in 8-bit precision
quantization_config = BitsAndBytesConfig(load_in_8bit=True)

# Load the text encoder model with quantization
text_encoder = T5EncoderModel.from_pretrained(
    "stabilityai/stable-diffusion-3-medium-diffusers",
    subfolder="text_encoder_3",
    quantization_config=quantization_config,
    torch_dtype=torch.float32  # Use float32 for CPU compatibility
)

# Load the Stable Diffusion pipeline with the default text encoder
pipe_default = StableDiffusion3Pipeline.from_pretrained(
    "stabilityai/stable-diffusion-3-medium-diffusers",
    torch_dtype=torch.float32,  # Use float32 for CPU compatibility
    low_cpu_mem_usage=True
)

# Integrate the custom text encoder into the pipeline
pipe_custom = StableDiffusion3Pipeline.from_pretrained(
    "stabilityai/stable-diffusion-3-medium-diffusers",
    torch_dtype=torch.float32,  # Use float32 for CPU compatibility
    low_cpu_mem_usage=True
)

# Replace the text encoder in the custom pipeline
pipe_custom.text_encoder = text_encoder

# Move the pipelines to CPU
pipe_default = pipe_default.to("cpu")
pipe_custom = pipe_custom.to("cpu")

# Define the prompt and generation parameters
prompt = "a photo of a cat holding a sign that says hello world"
negative_prompt = ""
num_inference_steps = 28
height = 512
width = 512
guidance_scale = 7.0

# Generate an image with the default text encoder
image_default = pipe_default(
    prompt=prompt,
    negative_prompt=negative_prompt,
    num_inference_steps=num_inference_steps,
    height=height,
    width=width,
    guidance_scale=guidance_scale,
).images[0]

# Generate an image with the custom text encoder
image_custom = pipe_custom(
    prompt=prompt,
    negative_prompt=negative_prompt,
    num_inference_steps=num_inference_steps,
    height=height,
    width=width,
    guidance_scale=guidance_scale,
).images[0]

# Save the images
image_default_path = "sd3_hello_world_default.png"
image_custom_path = "sd3_hello_world_custom.png"
image_default.save(image_default_path)
image_custom.save(image_custom_path)

# Display the images in the Jupyter notebook
fig, axs = plt.subplots(1, 2, figsize=(20, 10))

# Display default text encoder image
axs[0].imshow(image_default)
axs[0].axis('off')
axs[0].set_title("Default Text Encoder")

# Display custom text encoder image
axs[1].imshow(image_custom)
axs[1].axis('off')
axs[1].set_title("Custom Text Encoder")

plt.show()


/home/nimda/code/SF-T5-Example/.venv/lib/python3.10/site-packages/diffusers/models/transformers/transformer_2d.py:34: FutureWarning: `Transformer2DModelOutput` is deprecated and will be removed in version 1.0.0. Importing `Transformer2DModelOutput` from `diffusers.models.transformer_2d` is deprecated and this will be removed in a future version. Please use `from diffusers.models.modeling_outputs import Transformer2DModelOutput`, instead.
  deprecate("Transformer2DModelOutput", "1.0.0", deprecation_message)
`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/9 [00:00<?, ?it/s]

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

: 

# GPU More Memory Option

In [3]:
import os
from transformers import T5EncoderModel, BitsAndBytesConfig
from diffusers import StableDiffusion3Pipeline
import torch
from PIL import Image
import matplotlib.pyplot as plt

# Configure quantization to load the model in 8-bit precision
quantization_config = BitsAndBytesConfig(load_in_8bit=True)

# Load the text encoder model with quantization
text_encoder = T5EncoderModel.from_pretrained(
    "stabilityai/stable-diffusion-3-medium-diffusers",
    subfolder="text_encoder_3",
    quantization_config=quantization_config,
)

# Load the Stable Diffusion pipeline with the default text encoder
pipe_default = StableDiffusion3Pipeline.from_pretrained(
    "stabilityai/stable-diffusion-3-medium-diffusers",
    device_map="balanced",
    torch_dtype=torch.float16
)

# Check if sequential offloading is enabled
if any(module.enable_sequential_cpu_offload for _, module in pipe_default.components.items()):
    pipe_default = pipe_default.to("cpu")
else:
    pipe_default = pipe_default.to("cuda")

# Load the Stable Diffusion pipeline with the custom text encoder
pipe_custom = StableDiffusion3Pipeline.from_pretrained(
    "stabilityai/stable-diffusion-3-medium-diffusers",
    text_encoder=text_encoder,
    device_map="balanced",
    torch_dtype=torch.float16
)

# Check if sequential offloading is enabled
if any(module.enable_sequential_cpu_offload for _, module in pipe_custom.components.items()):
    pipe_custom = pipe_custom.to("cpu")
else:
    pipe_custom = pipe_custom.to("cuda")

# Define the prompt and generation parameters
prompt = "a photo of a goldren retriever holding a sign that says how about them apples"
negative_prompt = ""
num_inference_steps = 28
height = 1024
width = 1024
guidance_scale = 7.0

# Generate an image with the default text encoder
image_default = pipe_default(
    prompt=prompt,
    negative_prompt=negative_prompt,
    num_inference_steps=num_inference_steps,
    height=height,
    width=width,
    guidance_scale=guidance_scale,
).images[0]

# Generate an image with the custom text encoder
image_custom = pipe_custom(
    prompt=prompt,
    negative_prompt=negative_prompt,
    num_inference_steps=num_inference_steps,
    height=height,
    width=width,
    guidance_scale=guidance_scale,
).images[0]

# Save the images
image_default_path = "sd3_hello_world_default.png"
image_custom_path = "sd3_hello_world_custom.png"
image_default.save(image_default_path)
image_custom.save(image_custom_path)

# Display the images in the Jupyter notebook
fig, axs = plt.subplots(1, 2, figsize=(20, 10))

# Display default text encoder image
axs[0].imshow(image_default)
axs[0].axis('off')
axs[0].set_title("Default Text Encoder")

# Display custom text encoder image
axs[1].imshow(image_custom)
axs[1].axis('off')
axs[1].set_title("Custom Text Encoder")

plt.show()


`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/9 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

: 